![emart_img](https://media.discordapp.net/attachments/746440481629536321/753607541602254858/2020-09-10_10.18.28.png?width=1920&height=878)

In [1]:
import requests
import scrapy
from scrapy.http import TextResponse
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import re
import time
from fake_useragent import UserAgent

### url 확인

In [56]:
url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId=6000097651&page=2"
headers = {"user-agent": UserAgent().chrome}
req = requests.get(url_s, headers=headers)
req

<Response [200]>

### 카테고리별 url

In [3]:
# subcategory items

# 냉동상품
nd = ["6000097651","6000097652","6000097653","6000097640","6000097639","6000097702","6000097703","6000097704","6000097705","6000097706","6000097707"]

# 냉장식품
nj = ["6000097656","6000097658","6000097669","6000097671","6000097670"]

# 식용유/설탕/소금
oil = ['6000095598',"6000095711","6000095727","6000095709"]

# 장류/조미료
msg = ["6000095605","6000095607","6000095608","6000095604","6000095724","6000095606","6000095712","6000095726","6000095728","6000095729","6000095713","6000095714","6000095715"]

# 소스/양념/육수
src = ["6000095591","6000095590","6000095592","6000095594","6000095609","6000095725","6000095612","6000095593"]

# 밀가루/라면/믹스류
pw = ["6000095595","6000097134"]

# 편의식/가공기타
rt = ["6000097196","6000097637","6000097136","6000095508","6000097138","6000097137","6000095862","6000095506","6000096326"]

# 반찬류
kim = ["6000097506", "6000097507"]

# 과일
fr = ["6000095796", "6000095794"]

# 포장용기
pck = ["0006510363","0006510361","0006510364","0006510365","0006510366","0006510546"]

# 가게위생용품
cln = ["0006510362","0006510367"]

In [4]:
# main category items

# 유제품 69p
yjp = "6000095501"

# 야채/채소 135p
vg = "6000095740"

# 쌀/잡곡/견과 134p
rc = "6000095498"

# 수산/건어물 235p
fs = "6000095500"

# 축산/계란 200p
mt = "6000095499"

### 1) xpath 방식

#### 1-1 ) 카테고리 통합 크롤링
- 아이템 존재 여부에 따라 크롤링 페이지 번호 건너 뛰는 if문 미포함
- 2.4만개

In [59]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

# title
t = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[2]/div/a/em[1]/text()').extract()

# price
pr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[3]/div/em/text()').extract()

# min_gram

min_gram = re.findall('\d+.?\d?k?[g]', "".join(t))

#link
link = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[1]/div[2]/a/@href').extract()

# price per gram
ppr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[4]/div[@class="unit"]').extract()

len(t), len(pr), len(min_gram), len(ppr), len(link)

(80, 87, 46, 16, 80)

In [57]:
datas = {"link" : [1,1], "ppr" : ["a", "b"]}
pd.DataFrame(datas) 

,link,ppr
0,1,a
1,1,b


In [6]:
categories = [nd, nj, msg, src, pw, rt, kim, fr, pck, cln]
datas = []

for category in categories:
    for cate in category:
        for p in range(1, 140):
            url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
            headers = {"user-agent": UserAgent().chrome}
            req = requests.get(url_s, headers=headers)
            response = TextResponse(req.url, body=req.text, encoding="utf-8")

            t = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[2]/div/a/em[1]/text()').extract()
            pr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[3]/div/em/text()').extract()
            ppr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[4]/div/text()').extract()       
            link = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[1]/div[2]/a/@href').extract()
            
            idx = len(t)
            
            for i in range(idx):
                datas.append({
                    "category" : cate,
                    "title" : t[i],
                    "price" : pr[i],
                    "price_per_gram" : ppr,
                    "link" : link[i]
                })
                
df = pd.DataFrame(datas)
df.tail()

,category,title,price,price_per_gram,link
6723,6000095862,롯데 자일리톨화이트 81g,"3,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000032486448&siteNo...
6724,6000095862,[농심] 멘토스 미니 푸르티 150g,"3,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=0000008919045&siteNo...
6725,6000095862,[해태] 자두캔디 90g,"2,000","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000015615391&siteNo...
6726,6000095862,SCO_오리온 더 자일리톨 용기껌102g,"1,180","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000019328607&siteNo...
6727,6000095862,[이마트] 자일리톨 레인보우 껌 리필 105g,"2,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000010185386&siteNo...


In [52]:
categories = [yjp, vg, rc, fs, mt]
datas_b = []

for cate in categories:
    for p in range(1, 240):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
        headers = {"user-agent": UserAgent().chrome}
        req = requests.get(url_s, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        t = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[2]/div/a/em[1]/text()').extract()
        pr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[3]/div/em/text()').extract()
        ppr = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[2]/div[4]/div/text()').extract()       
        link = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[1]/div[2]/a/@href').extract()

        idx = len(t)

        for i in range(idx):
            datas_b.append({
                "category" : cate,
                "title" : t[i],
                "price" : pr[i],
                "price_per_gram" : ppr,
                "link" : link[i]
            })
                
df_2 = pd.DataFrame(datas_b)
df_2.tail()

,category,title,price,price_per_gram,link
17278,6000095500,제대로 말린 제수용 반건조 서대 2미 300g이상22~27cm,"79,000","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000043640207&siteNo...
17279,6000095500,지주식 남도 특품 곱창김 1톳(100장) 390g내외,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000037300270&siteNo...
17280,6000095500,[SSG머니 20만원 이상 구매시 1만원 지급][9/17(목) 순차출고] 동해 반건...,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000032699170&siteNo...
17281,6000095500,고향수산 흑산도 홍어 500g / 중간숙성,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615100&siteNo...
17282,6000095500,고향수산 흑산도 홍어 500g / 약한숙성,"43,500","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615103&siteNo...


In [7]:
df["min_gram"] = df["title"].map(lambda x: re.findall('\d+.?\d?k?[g]', x))
df.tail()

,category,title,price,price_per_gram,link,min_gram
6723,6000095862,롯데 자일리톨화이트 81g,"3,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000032486448&siteNo...,[81g]
6724,6000095862,[농심] 멘토스 미니 푸르티 150g,"3,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=0000008919045&siteNo...,[150g]
6725,6000095862,[해태] 자두캔디 90g,"2,000","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000015615391&siteNo...,[90g]
6726,6000095862,SCO_오리온 더 자일리톨 용기껌102g,"1,180","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000019328607&siteNo...,[102g]
6727,6000095862,[이마트] 자일리톨 레인보우 껌 리필 105g,"2,980","[(10g당:134원), (10g당:101원), (10g당:237원), (10g당:...",/item/itemView.ssg?itemId=1000010185386&siteNo...,[105g]


In [53]:
df_2["min_gram"] = df_2["title"].map(lambda x: re.findall('\d+.?\d?k?[g]', x))
df_2.tail()

,category,title,price,price_per_gram,link,min_gram
17278,6000095500,제대로 말린 제수용 반건조 서대 2미 300g이상22~27cm,"79,000","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000043640207&siteNo...,[300g]
17279,6000095500,지주식 남도 특품 곱창김 1톳(100장) 390g내외,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000037300270&siteNo...,[390g]
17280,6000095500,[SSG머니 20만원 이상 구매시 1만원 지급][9/17(목) 순차출고] 동해 반건...,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000032699170&siteNo...,[1kg]
17281,6000095500,고향수산 흑산도 홍어 500g / 중간숙성,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615100&siteNo...,[500g]
17282,6000095500,고향수산 흑산도 홍어 500g / 약한숙성,"43,500","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615103&siteNo...,[500g]


In [54]:
dfs = pd.concat([df, df_2]).reset_index(drop=True)
dfs.tail()

,category,title,price,price_per_gram,link,min_gram
24481,6000095500,제대로 말린 제수용 반건조 서대 2미 300g이상22~27cm,"79,000","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000043640207&siteNo...,[300g]
24482,6000095500,지주식 남도 특품 곱창김 1톳(100장) 390g내외,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000037300270&siteNo...,[390g]
24483,6000095500,[SSG머니 20만원 이상 구매시 1만원 지급][9/17(목) 순차출고] 동해 반건...,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000032699170&siteNo...,[1kg]
24484,6000095500,고향수산 흑산도 홍어 500g / 중간숙성,"20,900","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615100&siteNo...,[500g]
24485,6000095500,고향수산 흑산도 홍어 500g / 약한숙성,"43,500","[(100g당:1,504원), (100g당:5,429원), (100g당:6,311원...",/item/itemView.ssg?itemId=1000050615103&siteNo...,[500g]


### 2) css-selector 방식

#### 1-1 ) 카테고리 통합 크롤링
- 아이템 존재 여부에 따라 크롤링 페이지 번호 건너 뛰는 if문 포함
- 1.6만개

In [9]:
url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId=6000096986&page=1"
headers = {"user-agent": UserAgent().chrome}
req = requests.get(url_s, headers=headers)

dom = BeautifulSoup(req.content, "html.parser")
elements = dom.select('#ty_thmb_view > ul > li')
quit = dom.select_one('#area_itemlist')
len(quit)

4

In [10]:
categories = [nd, nj, oil, msg, src, pw, rt, kim, fr, pck, cln]
datas = []

for category in categories:
    print(category)
    for sub_category in category:
        for p in range(1, 140):
            url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
            headers = {"user-agent": UserAgent().chrome}
            response = requests.get(url_s, headers=headers)

            dom = BeautifulSoup(response.content, "html.parser")

            quit = dom.select_one('#area_itemlist')
            try:
                if len(quit) == 4:
                    elements = dom.select('#ty_thmb_view > ul > li')
                    for element in elements:
                        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
                        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
                        href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
                        link = [a['href'] for a in href.find_all('a', href=True)]
                        min_gram = re.findall('\d+.?\d?k?[g]', t)
                        try:
                            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
                        except:
                            ppr = "none"

                        datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})
            except:
                continue

df = pd.DataFrame(datas)
df.tail()

['6000097651', '6000097652', '6000097653', '6000097640', '6000097639', '6000097702', '6000097703', '6000097704', '6000097705', '6000097706', '6000097707']
['6000097656', '6000097658', '6000097669', '6000097671', '6000097670']
['6000095598', '6000095711', '6000095727', '6000095709']
['6000095605', '6000095607', '6000095608', '6000095604', '6000095724', '6000095606', '6000095712', '6000095726', '6000095728', '6000095729', '6000095713', '6000095714', '6000095715']
['6000095591', '6000095590', '6000095592', '6000095594', '6000095609', '6000095725', '6000095612', '6000095593']
['6000095595', '6000097134']
['6000097196', '6000097637', '6000097136', '6000095508', '6000097138', '6000097137', '6000095862', '6000095506', '6000096326']
['6000097506', '6000097507']
['6000095796', '6000095794']
['0006510363', '0006510361', '0006510364', '0006510365', '0006510366', '0006510546']
['0006510362', '0006510367']


,category,title,price,min_gram,price/gram,link
15977,0006510364,베네치아 라일락 러너,"20,000",[],none,/item/itemView.ssg?itemId=1000010076723&siteNo...
15978,0006510364,아라베스크 블랙 러너,"20,000",[],none,/item/itemView.ssg?itemId=1000010078165&siteNo...
15979,0006510364,"[로엔그린] PET 전용 투명컵리드 평면형 1,000개 (98파이)","20,000",[],none,/item/itemView.ssg?itemId=1000051202625&siteNo...
15980,0006510364,"[로엔그린] PET 전용 투명컵리드 평면형 1,000개 (92파이)","20,000",[],none,/item/itemView.ssg?itemId=1000050409028&siteNo...
15981,0006510364,에어플라워 에어홀더 500개(10/13온스/92파이용)_1BOX,"20,000",[],none,/item/itemView.ssg?itemId=1000032976272&siteNo...


In [11]:
categories = [yjp, vg, rc, fs, mt]
datas_b = []


for cate in categories:
    print(cate)
    for p in range(1, 240):
        url_b = "http://emart.ssg.com/disp/category.ssg?ctgId={}&page={}".format(cate, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_b, headers=headers)

        dom = BeautifulSoup(response.content, "html.parser")
 
        quit = dom.select_one('#area_itemlist')
        try:
            if len(quit) == 4:
                elements = dom.select('#ty_thmb_view > ul > li')
                for element in elements:
                    t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
                    pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
                    href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
                    link = [a['href'] for a in href.find_all('a', href=True)]
                    min_gram = re.findall('\d+.?\d?k?[g]', t)
                    try:
                        ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
                    except:
                        ppr = "none"

                    datas_b.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})
        except:
            continue
                
df_2 = pd.DataFrame(datas_b)
df_2.tail()

6000095501
6000095740
6000095498
6000095500
6000095499


,category,title,price,min_gram,price/gram,link
75,0006510367,리치팜데이너트 (28g*14입) x 3박스,"47,000",[28g],"100g당:3,997원",/item/itemView.ssg?itemId=1000035523738&siteNo...
76,0006510367,[머거본] 허니솔트믹스넛 160g 1봉,"4,500",[160g],"160g당:4,500원",/item/itemView.ssg?itemId=1000051573966&siteNo...
77,0006510367,건강담은 백년동안견과 7일분 20g X 7개입/유기샘,"22,900",[20g],"100g당:16,358원",/item/itemView.ssg?itemId=1000052475793&siteNo...
78,0006510367,2020년 햇 늘보리 쌀,"8,000",[],100g당:400원,/item/itemView.ssg?itemId=1000012078291&siteNo...
79,0006510367,[여량농협]참조은 혼합15곡1kg,"7,500",[15곡1kg],100g당:750원,/item/itemView.ssg?itemId=1000048839913&siteNo...


In [12]:
dfs = pd.concat([df, df_2]).reset_index(drop=True)
dfs.tail()

,category,title,price,min_gram,price/gram,link
16057,0006510367,리치팜데이너트 (28g*14입) x 3박스,"47,000",[28g],"100g당:3,997원",/item/itemView.ssg?itemId=1000035523738&siteNo...
16058,0006510367,[머거본] 허니솔트믹스넛 160g 1봉,"4,500",[160g],"160g당:4,500원",/item/itemView.ssg?itemId=1000051573966&siteNo...
16059,0006510367,건강담은 백년동안견과 7일분 20g X 7개입/유기샘,"22,900",[20g],"100g당:16,358원",/item/itemView.ssg?itemId=1000052475793&siteNo...
16060,0006510367,2020년 햇 늘보리 쌀,"8,000",[],100g당:400원,/item/itemView.ssg?itemId=1000012078291&siteNo...
16061,0006510367,[여량농협]참조은 혼합15곡1kg,"7,500",[15곡1kg],100g당:750원,/item/itemView.ssg?itemId=1000048839913&siteNo...


#### 1-2) 카테고리 통합 크롤링
- 크롤링 페이지번호 건너뛰는 if문 없음
- 1.3만개 데이터 수집

In [13]:
categories = [nd, nj, oil, msg, src, pw, rt, kim, fr, pck, cln]
datas = []

for category in categories:
    print(category)
    for sub_category in category:
        for p in range(1, 140):
            url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
            headers = {"user-agent": UserAgent().chrome}
            response = requests.get(url_s, headers=headers)
            dom = BeautifulSoup(response.content, "html.parser")
            elements = dom.select('#ty_thmb_view > ul > li')
            
            for element in elements:
                t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
                pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
                href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
                link = [a['href'] for a in href.find_all('a', href=True)]
                min_gram = re.findall('\d+.?\d?k?[g]', t)
                try:
                    ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
                except:
                    ppr = "none"
                datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})
                    
df = pd.DataFrame(datas)
df.tail()

['6000097651', '6000097652', '6000097653', '6000097640', '6000097639', '6000097702', '6000097703', '6000097704', '6000097705', '6000097706', '6000097707']
['6000097656', '6000097658', '6000097669', '6000097671', '6000097670']
['6000095598', '6000095711', '6000095727', '6000095709']
['6000095605', '6000095607', '6000095608', '6000095604', '6000095724', '6000095606', '6000095712', '6000095726', '6000095728', '6000095729', '6000095713', '6000095714', '6000095715']
['6000095591', '6000095590', '6000095592', '6000095594', '6000095609', '6000095725', '6000095612', '6000095593']
['6000095595', '6000097134']
['6000097196', '6000097637', '6000097136', '6000095508', '6000097138', '6000097137', '6000095862', '6000095506', '6000096326']
['6000097506', '6000097507']
['6000095796', '6000095794']
['0006510363', '0006510361', '0006510364', '0006510365', '0006510366', '0006510546']
['0006510362', '0006510367']


,category,title,price,min_gram,price/gram,link
13031,0006510362,기본형 회전형 디켄터 클리너 1개,"10,000",[],none,/item/itemView.ssg?itemId=1000034487719&siteNo...
13032,0006510362,[모노뜨] 코니퍼 린넨 키친타올,"10,000",[],none,/item/itemView.ssg?itemId=1000029729357&siteNo...
13033,0006510362,다용도 90cm 방수매트 국내산 키친아트 김장매트,"10,020",[],none,/item/itemView.ssg?itemId=1000050790056&siteNo...
13034,0006510362,20매(소) 태화)니트릴 요리쿡장갑,"10,020",[],none,/item/itemView.ssg?itemId=1000050790280&siteNo...
13035,0006510362,고무장갑 태화 일반가사 라텍스 특대_X2 빨강,"10,020",[],none,/item/itemView.ssg?itemId=1000050791574&siteNo...


In [14]:
categories = [yjp, vg, rc, fs, mt]
datas_b = []


for cate in categories:
    for p in range(1, 250):
        url_b = "http://emart.ssg.com/disp/category.ssg?ctgId={}&page={}".format(cate, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_b, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
            link = [a['href'] for a in href.find_all('a', href=True)]            
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"
            datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})     

df_2 = pd.DataFrame(datas_b)
df_2.tail()

,category,title,price,min_gram,price/gram,link
418,6000095498,매일 좋은 견과 800g x15개 간식 호프 메뉴 식당 마른,"315,190",[800g],none,/item/itemView.ssg?itemId=1000052935841&siteNo...
419,6000095498,매일 좋은 견과 베리 요거트 800g x15개 간식 호프,"315,190",[800g],none,/item/itemView.ssg?itemId=1000052935845&siteNo...
420,6000095498,볶음 사차인지 800g x15개 간식 호프 메뉴 식당 마른,"330,190",[800g],none,/item/itemView.ssg?itemId=1000052935870&siteNo...
421,6000095498,국산 볶음 땅콩 15kg,"353,860",[15kg],none,/item/itemView.ssg?itemId=1000052935746&siteNo...
422,6000095498,아몬드 볶음 1kg x15개 간식 호프 메뉴 식당 마른,"372,140",[1kg],none,/item/itemView.ssg?itemId=1000052935884&siteNo...


In [15]:
dfs = pd.concat([df, df_2]).reset_index(drop=True)
dfs.tail()

,category,title,price,min_gram,price/gram,link
13454,6000095498,매일 좋은 견과 800g x15개 간식 호프 메뉴 식당 마른,"315,190",[800g],none,/item/itemView.ssg?itemId=1000052935841&siteNo...
13455,6000095498,매일 좋은 견과 베리 요거트 800g x15개 간식 호프,"315,190",[800g],none,/item/itemView.ssg?itemId=1000052935845&siteNo...
13456,6000095498,볶음 사차인지 800g x15개 간식 호프 메뉴 식당 마른,"330,190",[800g],none,/item/itemView.ssg?itemId=1000052935870&siteNo...
13457,6000095498,국산 볶음 땅콩 15kg,"353,860",[15kg],none,/item/itemView.ssg?itemId=1000052935746&siteNo...
13458,6000095498,아몬드 볶음 1kg x15개 간식 호프 메뉴 식당 마른,"372,140",[1kg],none,/item/itemView.ssg?itemId=1000052935884&siteNo...


#### 2-1) 카테고리 분리 크롤링
- 아이템 존재 여부에 따라 크롤링 페이지 번호 건너 뛰는 if문 포함

#### 2-2) 카테고리 분리 크롤링
- 크롤링 페이지번호 건너뛰는 if문 없음
- link 수집 코드 없음
- 1.4만개

In [16]:
# 냉동식품

categories = nd
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})

df = pd.DataFrame(datas)
df.tail()

,category,title,price,min_gram,price/gram
3933,6000097707,바사삭 치킨! 전상품 할인 진행중~,"6,980",[],none
3934,6000097707,캐빈디쉬 해쉬브라운 1.2kg,"7,200",[1.2kg],none
3935,6000097707,[사옹원] 부침명장 한입아삭김치전 300g(30gx10개),"13,900","[300g, 30g]",none
3936,6000097707,[사옹원] 튀김공방 매콤오징어 튀김 외 골라담기,"14,900",[],none
3937,6000097707,쏘굿 수제 명절 튀김 세트 구성 5호부터 1호까지,"22,900",[],none


In [17]:
# 냉장식품

categories = nj
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
            
df_2 = pd.DataFrame(datas)
df_2.tail()

,category,title,price,min_gram,price/gram
721,6000097656,부산의어묵 맛뜰안세트 2호 (어묵탕+떡볶이전용),"107,970",[],none
722,6000097656,맛뜰안어묵 실속세트 2호-프리미엄,"108,680",[],none
723,6000097656,맛뜰안어묵 실속세트 3호-로얄,"192,340",[],none
724,6000097656,간편 즉석 조리식품 구운어묵 360g X 25_1 EA,"194,190",[360g],none
725,6000097656,식재료 부산미도 매운맛 당면어묵 350gX35,"267,800",[350g],none


In [18]:
# 식용유 등

categories = oil
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_3 = pd.DataFrame(datas)
df_3.tail()

""


In [19]:
# 조미료

categories = msg
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_4 = pd.DataFrame(datas)
df_4.tail()

""


In [20]:
# 소스

categories = src
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})

df_5 = pd.DataFrame(datas)
df_5.tail()

""


In [21]:
# 밀가루 등

categories = pw
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_6 = pd.DataFrame(datas)
df_6.tail()

""


In [22]:
# 간편식

categories = rt
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_7 = pd.DataFrame(datas)
df_7.tail()

,category,title,price,min_gram,price/gram
1595,6000095508,다질리언 홍차-아쌈 8티백(큐리오틴)/아쌈티백,"12,570",[],none
1596,6000095508,다질리언 홍차-잉글리쉬애프터눈 8티백(큐리오틴),"12,570",[],none
1597,6000095508,쟈뎅 바리스타 수아베 원두커피 500g,"9,200",[500g],none
1598,6000095508,갓볶은메가커피 코스타리카 따라주 500g,"15,400",[500g],none
1599,6000095508,아크바 얼그레이 홍차라떼 스틱 1곽 12개입,"6,250",[],none


In [23]:
# 김치

categories = kim
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_8 = pd.DataFrame(datas)
df_8.tail()

""


In [24]:
# 과일

categories = fr
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_9 = pd.DataFrame(datas)
df_9.tail()

""


In [25]:
# 포장용기

categories = pck
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_10 = pd.DataFrame(datas)
df_10.tail()

,category,title,price,min_gram,price/gram
5451,0006510366,크린랩 알뜰 지퍼백 25cmX30cm 50매 1박스 20개,"131,790",[],none
5452,0006510366,크린랩(22X50) 40개 1box,"147,000",[],1m당:74원
5453,0006510366,크린랩(30X50) 30개 1box,"151,200",[],1m당:101원
5454,0006510366,크린랩 랩 22cmX100M 1박스 30개,"193,730",[],none
5455,0006510366,Photato Fry Paper 1팩(1000개),"400,000",[],none


In [26]:
# 주방위생용품

categories = cln
datas = []

for sub_category in categories:
    for p in range(1, 140):
        url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, p)
        headers = {"user-agent": UserAgent().chrome}
        response = requests.get(url_s, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.select('#ty_thmb_view > ul > li')

        for element in elements:
            t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
            pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
            min_gram = re.findall('\d+.?\d?k?[g]', t)
            try:
                ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
            except:
                ppr = "none"

            datas.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})
                    
df_11 = pd.DataFrame(datas)
df_11.tail()

""


In [27]:
# 유제품

cate = yjp
datas_b = []


for p in range(1, 70):
    url_b = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"

        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})     

            
df_12 = pd.DataFrame(datas_b)
df_12.tail()

""


In [28]:
# 야채/채소

cate = vg
datas_b = []


for p in range(1, 136):
    url_b = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"

        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})     

df_13 = pd.DataFrame(datas_b)
df_13.tail()

""


In [29]:
# 쌀/잡곡/견과

cate = rc
datas_b = []


for p in range(1, 135):
    url_b = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"

        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})     

df_14 = pd.DataFrame(datas_b)
df_14.tail()

""


In [30]:
# 수산/건어물

cate = fs
datas_b = []


for p in range(1, 236):
    url_b = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"

        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})     

df_15 = pd.DataFrame(datas_b)
df_15.tail()

,category,title,price,min_gram,price/gram
1675,6000095500,[9월 17일부터 순차출고][조선호텔] 명품 법성포 전통 재래굴비 VIP 3.0kg,"2,500,000",[3.0kg],"100g당:83,334원"
1676,6000095500,"[마칸] 자숙 흰다리새우 5kg 1박스 대 (300마리,찐새우)","99,000",[5kg],"100g당:1,980원"
1677,6000095500,기장프리미엄 꼭지미역다시마세트2호,"47,900",[],none
1678,6000095500,제주 옥돔(대) 1kg(4마리),"76,000",[1kg],"100g당:7,600원"
1679,6000095500,[9/17(목) 순차출고] 완도 참전복세트 2호(1.0kg/9~10미),"54,000",[1.0kg],"100g당:5,400원"


In [31]:
# 축산/계란

cate = mt
datas_b = []


for p in range(1, 200):
    url_b = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(cate, p)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"

        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr})     

df_16 = pd.DataFrame(datas_b)
df_16.tail()

,category,title,price,min_gram,price/gram
715,6000095499,[햇살닭]저염훈제 닭가슴살플러스 3kg(100gX30팩),"60,000","[3kg, 100g]",none
716,6000095499,[햇살닭] 흑마늘닭가슴살소시지 3kg(100gX30팩),"60,000","[3kg, 100g]",none
717,6000095499,[햇살닭] 포테이토닭가슴살소시지 3kg(100gX30팩),"60,000","[3kg, 100g]",none
718,6000095499,[맛닭] 훈제 닭가슴살 스테이크 (매콤한맛) (100gx50팩),"60,000",[100g],none
719,6000095499,[햇살닭] 천사섬함초닭가슴살소시지 3kg(100gX30팩),"60,000","[3kg, 100g]",none


In [32]:
dfs = pd.concat([df, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_14, df_15, df_16]).reset_index(drop=True)
dfs

,category,title,price,min_gram,price/gram
0,6000097653,노브랜드 쫄깃한 베이컨 300 g,"3,980",[300 g],none
1,6000097653,[CJ] 햄스빌 베이컨 (110gx3),"7,980",[110g],10g당:242원
2,6000097653,[청정원] 담백한 베이컨405g(135g*3),"6,980","[405g, 135g]",10g당:173원
3,6000097653,cj햄스빌 굿베이컨 130g*3,"6,980",[130g],10g당:179원
4,6000097653,[피코크] 리얼 오리지날 베이컨 300g (100g*3),"6,680","[300g, 100g]",10g당:223원
...,...,...,...,...,...
14115,6000095499,[햇살닭]저염훈제 닭가슴살플러스 3kg(100gX30팩),"60,000","[3kg, 100g]",none
14116,6000095499,[햇살닭] 흑마늘닭가슴살소시지 3kg(100gX30팩),"60,000","[3kg, 100g]",none
14117,6000095499,[햇살닭] 포테이토닭가슴살소시지 3kg(100gX30팩),"60,000","[3kg, 100g]",none
14118,6000095499,[맛닭] 훈제 닭가슴살 스테이크 (매콤한맛) (100gx50팩),"60,000",[100g],none


#### 3) 크롤링 함수 이용
- 크롤링 페이지번호 건너뛰는 if문 없음
- 크롤링 안 됨..

In [33]:
def emart_subcate_items(sub_category, page):
    url_s = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(sub_category, page)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_s, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')
    datas_s = []

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
        link = [a['href'] for a in href.find_all('a', href=True)]            

        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"
        datas_s.append({"category":sub_category, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})
    
    return pd.DataFrame(datas_s)

In [34]:
def emart_maincate_items(cate, page):
    url_b = "http://emart.ssg.com/disp/category.ssg?ctgId={}&page={}".format(cate, page)
    headers = {"user-agent": UserAgent().chrome}
    response = requests.get(url_b, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select('#ty_thmb_view > ul > li')
    datas_b = []

    for element in elements:
        t = element.select_one('div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko').text
        pr = element.select_one('div.cunit_info > div.cunit_price.notranslate > div > em').text
        href = element.select_one('div.cunit_info > div.cunit_md.notranslate > div')
        link = [a['href'] for a in href.find_all('a', href=True)]            
        
        min_gram = re.findall('\d+.?\d?k?[g]', t)
        try:
            ppr = element.select_one('div.cunit_info > div.cunit_prw > div').text.replace("(", "").replace(")", "")
        except:
            ppr = "none"
        datas_b.append({"category":cate, "title":t, "price":pr, "min_gram":min_gram, "price/gram":ppr, "link":link[0]})     
    
    return pd.DataFrame(datas_b)

In [35]:
dfs = pd.DataFrame()
categories = [nd, nj, oil, msg, src, pw, rt, kim, fr, pck, cln]

for category in categories:
    for sub_category in category:
        for page in range(1, 140):
            print(page, end=" ")
            df = emart_subcate_items(sub_category, page)
            dfs.append(df)
        time.sleep(2)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38

50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84

94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 12

129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 

In [36]:
dfs_2 = pd.DataFrame()
categories = [yjp, vg, rc, fs, mt]

for cate in categories:
    print(cate)
    for page in range(1, 250):
        print(page, end=" ")
        df2 = emart_maincate_items(cate, page)
        dfs_2.append(df2)
    print()
    time.sleep(2)

6000095501
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 
6000095740
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33

In [37]:
len(dfs), len(dfs_2)

(0, 0)

In [38]:
emart_items = pd.concat([dfs, dfs_2])
emart_items

""


In [146]:
dfs

""


In [47]:
emart_subcate_items("6000097651", 1)

,category,title,price,min_gram,price/gram,link
0,6000097651,[CJ] 더(The)건강한햄 슬라이스 90g,"1,980",[90g],10g당:220원,/item/itemView.ssg?itemId=0000008038256&siteNo...
1,6000097651,[노브랜드] 리얼 햄 1kg,"6,980",[1kg],10g당:70원,/item/itemView.ssg?itemId=1000018629216&siteNo...
2,6000097651,[목우촌] 주부9단 김밥햄 340g(170g*2개입),"4,980","[340g, 170g]",10g당:147원,/item/itemView.ssg?itemId=0000007889123&siteNo...
3,6000097651,CJ 더건강한등심슬라이스250g,"5,980",[250g],10g당:240원,/item/itemView.ssg?itemId=1000010640290&siteNo...
4,6000097651,[롯데] 의성마늘햄 180g*3,"7,580",[180g],10g당:141원,/item/itemView.ssg?itemId=0000008121013&siteNo...
...,...,...,...,...,...,...
75,6000097651,[CJ]더건강한햄150g3,"4,400",[150g],10g당:98원,http://earlymorning.ssg.com/item/itemView.ssg?...
76,6000097651,CJ 햄스빌 바로먹는햄 소고기120g,"3,980",[120g],10g당:332원,/item/itemView.ssg?itemId=1000052010007&siteNo...
77,6000097651,비프 파스트라미 100g,"4,980",[100g],none,http://earlymorning.ssg.com/item/itemView.ssg?...
78,6000097651,CJ 햄스빌 바로먹는햄 목살120g,"3,980",[120g],10g당:332원,/item/itemView.ssg?itemId=1000052010003&siteNo...


In [40]:
emart_maincate_items(yjp, 1)

,category,title,price,min_gram,price/gram,link
0,6000095501,[서울우유] 서울우유 2.3L,"5,480",[],100ml당:239원,/item/itemView.ssg?itemId=0000007095585&siteNo...
1,6000095501,[피코크] 우유(1등급) 1000ml,"1,880",[],100ml당:188원,/item/itemView.ssg?itemId=1000022568383&siteNo...
2,6000095501,[서울우유] 서울우유 1000ml 나100%,"2,560",[],100ml당:256원,/item/itemView.ssg?itemId=0000006615474&siteNo...
3,6000095501,[서울우유] 커피 삼각 포리 (200ml*3개),"1,980",[],100ml당:330원,/item/itemView.ssg?itemId=0000007095596&siteNo...
4,6000095501,[노브랜드] 마시는 요구르트 65ml * 15개입,980,[],10ml당:11원,/item/itemView.ssg?itemId=1000017265141&siteNo...
...,...,...,...,...,...,...
75,6000095501,[매일]요미요미 유기농주스 사과배포도 (125ML*3),"2,900",[],100ml당:774원,/item/itemView.ssg?itemId=1000005526401&siteNo...
76,6000095501,매일 저지방 1% 900mL,"2,480",[],100ml당:276원,/item/itemView.ssg?itemId=1000026434368&siteNo...
77,6000095501,서울 딸기우유 200ml*3입,"2,390",[],100ml당:399원,/item/itemView.ssg?itemId=1000028249270&siteNo...
78,6000095501,[매일] 오리지널(200ml*3개) 600ml,"1,980",[],100ml당:330원,/item/itemView.ssg?itemId=0000007904387&siteNo...
